## Train Script for Bengali AI models
Load in model classes and processing scripts. Run training and evaluation here

In [12]:
# Packages
import torch
import torchvision
from torch.utils.data.sampler import WeightedRandomSampler
from torch.utils.data import DataLoader
from torch.autograd import Variable


from model.modelBase import *
from model.wrapperModel import *
from utils.evalUtils import *
from ProcessAndAugment import *

In [2]:
# paths
datadir = "./data"
inputdir= datadir + "/raw"
outputdir= datadir + "/processed"

In [3]:
# Parameters
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size=64

n_grapheme = 168
n_vowel = 11
n_consonant = 7
n_total = n_grapheme + n_vowel + n_consonant
print('n_total', n_total)

n_total 186


In [4]:
# Model Selection

# core model
predictor = densenet(in_channels=1, out_dim=n_total)
print('predictor', type(predictor))

# select our wrapper class
classifier = BengaliClassifier(predictor)
print('classifier',type(classifier))

predictor <class 'model.modelBase.densenet'>
classifier <class 'model.wrapperModel.BengaliClassifier'>


In [5]:
# Model Parameters
epochs = 10
lr = .001 # TODO: starting with flat LR, but need to implement scheduler
bs = 64

optimizer = torch.optim.Adam(classifier.parameters(), lr=lr)

# ignoring scheduler for now until we have baseline
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.7, patience=5, min_lr=1e-10)


validate_every = 5 # TODO: validate every n batches or epochs
checkpoint_every = 5 # TODO: implement model checkpoints

## Prep Data
Utilizes our process and data augmentation script

In [6]:
# load train file and generate dataset
train = pd.read_csv(datadir+'/train.csv')
indices = [0] # just set to list of all indices when actually training
dataset, crop_rsz_img = genDataset(indices, inputdir, train = train) # generates the dataset class

print(dataset.get_example(0))


image_df_list 1
(array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([15,  9,  5], dtype=int64))


In [7]:
# our weights for the weighted random sampler for each epoch
consonant_weights = genWeightTensor("consonant_diacritic", train[:len(crop_rsz_img)])
root_weights = genWeightTensor("grapheme_root", train[:len(crop_rsz_img)])
vowel_weights = genWeightTensor("vowel_diacritic", train[:len(crop_rsz_img)])
grapheme_weights = genWeightTensor("grapheme", train[:len(crop_rsz_img)])

weights = {"consonant_diacritic": consonant_weights,
           "grapheme_root": root_weights,
           "vowel_diacritic": vowel_weights,
           "grapheme": grapheme_weights}

weight_keys = list(weights.keys())

## Training

In [15]:
for i, wkey in zip(range(epochs), itertools.cycle(weight_keys)):
    print(i, wkey)
    
    # generate sampler and loader specific to epoch
    wgt_val = weights[wkey]
    sampler = WeightedRandomSampler(wgt_val, len(wgt_val))
    train_loader = DataLoader(dataset, batch_size=bs, sampler=sampler)
    num_batches = len(train_loader)
    
    # init
    predictor.train()
    classifier.train()
    
    for j, (images, labels) in enumerate(train_loader):
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)
        
        # reset
        # since we are doing a bunch of classes - do we have to zero out all of them
        predictor.zero_grad()
        classifier.zero_grad()
        
        # run model
        loss, metrics, pred = classifier(images)
        
        # compute loss and step
        loss = criterion(L1_loss, gs)
        loss.backward()
        
        optimizer.step()
        
        print(metrics)

0 consonant_diacritic


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [3, 1, 3, 3], but got 3-dimensional input of size [64, 128, 128] instead